In [1]:
from __future__ import division
import numpy as np
from numpy import linalg as LA
#np.seterr(divide='ignore') # these warnings are usually harmless for this code
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
import os
import scipy.stats as stats
import pyhsmm
from pyhsmm.util.text import progprint_xrange
import pyhsmm.basic.distributions as distributions
import scipy.io as sio
import csv
import copy
import time
import pickle
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table, MetaData, Column, Integer, String
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sklearn import preprocessing

In [ ]:
filename = 'data_devices_trip.sav'
data_devices_trip = pickle.load(open(filename, 'rb'))

In [2]:
#EEFECTS: return new data in form: data = {} and data[device]={"trip":[]}
def dataTransform(data_devices):
    data = {}
    for i, devi in enumerate(data_devices):
        #print(i, devi)
        data[devi] = {}
        for ii in range(data_devices[devi].shape[0]):
            data_temp = data_devices[devi][ii]
            trip = int(data_temp[0])
            speed = data_temp[1]
            acc = data_temp[2]
            try:
                data[devi][trip].append([speed,acc])
            except:
                data[devi][trip] = []
                data[devi][trip].append([speed,acc])
    return data

In [3]:
# get data_devices_trip = {} and data_devices_trip[device]={"trip":[]}
filename = 'data_devices.sav'
data_devices = pickle.load(open(filename, 'rb'))
data_devices_trip = dataTransform(data_devices)
#another way to get data_devices_trip, but this way is a little bit slow
#filename = 'data_devices_trip.sav'
#data_devices_trip = pickle.load(open(filename, 'rb'))

In [ ]:



posteriormodels = {}
i = 0
for devi, value1 in data_devices_trip.items() :
#for i, devi in enumerate(data_devices):
    print('devi', devi)
    if(len(data_devices_trip[devi]) == 0):
            print('oops, this is a none set')
            continue
    else:
            posteriormodels[devi]={}
            
    for trip,value2 in data_devices_trip[devi].items():
        print('trip',trip)
        data_trip = np.array(data_devices_trip[devi][trip])
        
        data_scaled = preprocessing.scale(data_trip)#implement data normalization 
        Nmax = 200 # preset the maximum states
        # and some hyperparameters
        obs_dim =  data_scaled.shape[1] # data dimensions
        obs_hypparams = {'mu_0':np.zeros(np.int(obs_dim)),
                        'sigma_0':np.eye(np.int(obs_dim)),
                        'kappa_0':0.25, # 0.2 5
                        'nu_0':obs_dim+2}
        # Define the observation distribution
        obs_distns = [pyhsmm.distributions.Gaussian(**obs_hypparams) for state in range(Nmax)]

        # Define the posterior inference model
        posteriormodels[devi][trip] = pyhsmm.models.WeakLimitStickyHDPHMM(
            kappa=6.,alpha=1.,gamma=1.,init_state_concentration=1.,
            obs_distns=obs_distns)

        # Sampling process, for 100 round
        Sampling_step = 100
        Sampling_xaxis = range(1,Sampling_step+1)

        # Add the data to the model and train
        posteriormodels[devi][trip].add_data(data_scaled)
        Meth2_LLH = np.zeros((Sampling_step,1))

        # Sampling process, for 100 around
        for idx in progprint_xrange(Sampling_step):
            posteriormodels[devi][trip].resample_model()
            #Meth2_LLH[idx] = posteriormodel.log_likelihood()
    i = i + 1   
    if i == 6:
        break
        
# save the model to disk
filename = 'posterior_models_test.sav'
pickle.dump(posteriormodels, open(filename, 'wb'))

In [ ]:
posteriormodels = {}
i = 0
for devi, value1 in data_devices_trip.items() :
#for i, devi in enumerate(data_devices):
    print('devi', devi)
    if(len(data_devices_trip[devi]) == 0):
            print('oops, this is a none set')
            continue
    else:
            posteriormodels[devi]={}
    i = i + 1   
    if i == 6:
        break            
    